In [2]:
import os
from pyspark.sql import SparkSession

#spark = SparkSession.builder.master("local").appName("mllib_classifier").getOrCreate()
spark = SparkSession.builder.appName("StockPricePrediction").getOrCreate()

# Load training data
filename = "KOv2.csv"
data = spark.read.csv(filename,  inferSchema=True, header = True)
#data.show(2)

In [2]:
data.describe

<bound method DataFrame.describe of DataFrame[Date: int, Open: double, High: double, Low: double, Close: double, AdjClose: double, Volume: int, MarketCap: double, EnterpriseValue: double, PeRatio: double, PsRatio: double, PbRatio: double, EnterprisesValueRevenueRatio: double, EnterprisesValueEBITDARatio: double, TotalRevenue: bigint, OperatingRevenue: bigint, CostOfRevenue: bigint, GrossProfit: bigint, OperatingExpense: bigint, SellingGeneralAndAdministration: bigint, OperatingIncome: bigint, OtherIncomeExpense: bigint, PretaxIncome: bigint, TaxProvision: bigint, NetIncomeCommonStockholders: bigint, NetIncome: bigint, NetIncomeIncludingNoncontrollingInterests: bigint, NetIncomeContinuousOperations: bigint, DilutedNIAvailtoComStockholders: bigint, BasicEPS: double, DilutedEPS: double, BasicAverageShares: bigint, DilutedAverageShares: bigint, TotalOperatingIncomeAsReported: bigint, TotalExpenses: bigint, NetIncomeFromContinuingAndDiscontinuedOperation: bigint, NormalizedIncome: double, E

In [6]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("SubsetColumns").getOrCreate()

# Assuming you have an existing DataFrame called "original_df"
# Replace "original_df" with your actual DataFrame name
original_df = data  # Replace this with your actual DataFrame

# Define a list of columns you want to include in the subset
selected_columns = ["AdjClose", "PeRatio", "PsRatio", "PbRatio","TotalRevenue","GrossProfit"]

# Use the select function to create a new DataFrame with the subset of columns
new_df = original_df.select(selected_columns)

# Show the new DataFrame
new_df.show()


23/11/21 02:10:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
+---------+-------+-------+-------+------------+-----------+
| AdjClose|PeRatio|PsRatio|PbRatio|TotalRevenue|GrossProfit|
+---------+-------+-------+-------+------------+-----------+
|56.490002| 23.228|  5.509|  9.305| 11953000000| 7296000000|
|61.442528| 26.529|  6.019| 10.326| 11972000000| 7060000000|
|63.164001| 28.324|  6.275|  11.13| 10980000000| 6663000000|
|  59.9179| 27.899|  6.537| 12.072| 10125000000| 5612000000|
|58.069668| 25.464|  5.898| 10.529| 11063000000| 6497000000|
|61.810516| 26.544|  6.815| 10.977| 11325000000| 6495000000|
|61.787926| 27.556|  6.961| 11.686| 10491000000| 6400000000|
|57.901939| 29.167|   6.79| 11.547|  9464000000| 5376000000|
|53.086636| 28.059|  6.239|  10.18| 10042000000| 6065000000|
|53.305889| 32.401|  6.999| 11.462| 10129000000| 6342000000|
|50.077721| 29.447|  6.902|  11.77|  9020000000| 5515000000|
|44.300453| 28.415|   7.08

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create a Spark session
spark = SparkSession.builder.appName("StringToFloatConversion").getOrCreate()

# Assuming you have a DataFrame called "your_data"
# Replace "your_data" with your actual DataFrame name
data = data  # Replace this with your actual DataFrame

# Get the list of columns in the DataFrame
all_columns = data.columns

# Iterate through each column and update its type to float
for column in all_columns:
    data = data.withColumn(column, col(column).cast("float"))

# Show the updated DataFrame
data.describe


In [7]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
#from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create a Spark session
#spark = SparkSession.builder.appName("StockPricePrediction").getOrCreate()

# Assuming your data is in a DataFrame called "data"
# Replace "target_column" with the actual column name of your target variable
target_column = 'AdjClose'
features_columns = ['PeRatio','PsRatio','PbRatio','TotalRevenue','GrossProfit']

# Assemble the features into a vector
assembler = VectorAssembler(inputCols=features_columns, outputCol="features")
data_assembled = assembler.transform(new_df)

# Split the data into training and testing sets
(train_data, test_data) = data_assembled.randomSplit([0.8, 0.2], seed=123)

# Initialize the logistic regression model
lr = LinearRegression(labelCol=target_column, featuresCol="features",
                       maxIter=10,
                      regParam=0.3, 
                      elasticNetParam=0.8)

# Create a pipeline with the assembler and the logistic regression model
#pipeline = Pipeline(stages=[lr])

# Train the model
model = lr.fit(train_data)

# Print the weights and intercept for linear regression
print("Weights: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))

# Make predictions on the test set
#predictions = model.transform(test_data)

# Evaluate the model using a binary classification evaluator
#evaluator = BinaryClassificationEvaluator(labelCol=target_column)
#accuracy = evaluator.evaluate(predictions)

#print(f"Accuracy: {accuracy}")

# Optionally, you can print the coefficients of the logistic regression model
#print(f"Coefficients: {model.stages[-1].coefficients}")


Weights: [0.025478994978851217,3.311269724942866,-0.07299225835558495,4.085880907200866e-09,0.0]
Intercept: -24.051203190308343


In [8]:
from pyspark.ml.evaluation import RegressionEvaluator

# compute predictions. this will append column "prediction" to dataframe
lrPred = model.transform(test_data)
lrPred.show(10)

ev = RegressionEvaluator(predictionCol="prediction", labelCol="AdjClose")

print('-'*20)
print("METRICS")
print("Mean Squared Error:", ev.evaluate(lrPred, {ev.metricName: "mse"}))
print("R Squared:", ev.evaluate(lrPred, {ev.metricName:'r2'}))

+---------+-------+-------+-------+------------+-----------+--------------------+------------------+
| AdjClose|PeRatio|PsRatio|PbRatio|TotalRevenue|GrossProfit|            features|        prediction|
+---------+-------+-------+-------+------------+-----------+--------------------+------------------+
| 1.957785| 22.991|  3.016|  9.249|  2248900000| 1280400000|[22.991,3.016,9.2...|-4.964993951648665|
|   2.6057|  19.81|  2.595|  9.694|  2739300000| 1695200000|[19.81,2.595,9.69...|-4.468852746954269|
| 4.719271| 32.788|   4.27| 14.041|  3056000000| 2051000000|[32.788,4.27,14.0...|2.3848915753993474|
| 4.833506| 29.348|  4.315| 11.564|  3507500000| 2211200000|[29.348,4.315,11....| 4.471828023842505|
| 5.089425| 23.897|  3.761| 10.921|  3352000000| 2201000000|[23.897,3.761,10....|1.9100781356473462|
| 6.481538| 27.394|  4.319| 11.724|  4017000000| 2639000000|[27.394,4.319,11....| 6.505364707435547|
|11.000153|   27.4|  5.564|   9.18|  4795000000| 3094000000|[27.4,5.564,9.18,...|13.9925560